# rectus femoris muscle



In [1]:
# Step 1: Mount Google Drive and Install Libraries

# --- 1.1: Mount Google Drive ---
# This will prompt you for authorization.
from google.colab import drive
drive.mount('/content/drive')

# --- 1.2: Install Segmentation Models Pytorch library ---
# This library contains many pre-built models like U-Net, U-Net++, Attention U-Net, etc.
!pip install segmentation-models-pytorch -q

# --- 1.3: Import all necessary libraries ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from PIL import Image
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Imports for the model and transformations
import segmentation_models_pytorch as smp
import torchvision.transforms.functional as F
from torchvision import transforms
from sklearn.model_selection import train_test_split

print("Libraries installed and imported successfully!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
Libraries installed and imported successfully!


In [18]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# ========================================================================
# This is the simplified Dataset class without transformations
# ========================================================================

class UltrasoundNpyDataset_NoTransforms(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        image_np = self.x_data[idx]
        mask_np = self.y_data[idx]

        # Normalize image to [0, 1] based on min/max of the dataset
        image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min() + 1e-6)
        image_tensor = torch.from_numpy(image_np).float()
        if image_tensor.ndim == 3 and image_tensor.shape[-1] in [1, 3]:
            image_tensor = image_tensor.permute(2, 0, 1)
        elif image_tensor.ndim == 2:
            image_tensor = image_tensor.unsqueeze(0)

        # Process mask: ensure it’s 2D (H, W) and convert to binary float
        if mask_np.ndim == 3 and mask_np.shape[-1] == 1:
            mask_np = np.squeeze(mask_np, axis=-1)  # Remove the last dimension if it’s 1
        mask_np = (mask_np > 0).astype(np.float32)  # Binary mask (0 or 1)
        mask_tensor = torch.from_numpy(mask_np).float()
        if mask_tensor.ndim == 2:
            mask_tensor = mask_tensor.unsqueeze(0)  # Add channel dimension to [1, H, W]

        return image_tensor, mask_tensor


# ========================================================================
# The rest of your script, now using the new Dataset class
# ========================================================================

# --- Step 1: Define File Paths ---
# This path points to the folder containing your data.
DATA_FOLDER = '/content/drive/MyDrive/intern RF transverse latest file/'

X_TRAIN_PATH = os.path.join(DATA_FOLDER, 'X_train.npy')
Y_TRAIN_PATH = os.path.join(DATA_FOLDER, 'y_train.npy')
X_VAL_PATH = os.path.join(DATA_FOLDER, 'X_val.npy')
Y_VAL_PATH = os.path.join(DATA_FOLDER, 'y_val.npy')
X_TEST_PATH = os.path.join(DATA_FOLDER, 'X_test.npy')
Y_TEST_PATH = os.path.join(DATA_FOLDER, 'y_test.npy')


# --- Step 2: Load Data and Create DataLoaders ---
print("Loading pre-split data...")
x_train = np.load(X_TRAIN_PATH)
y_train = np.load(Y_TRAIN_PATH)
x_val = np.load(X_VAL_PATH)
y_val = np.load(Y_VAL_PATH)
x_test = np.load(X_TEST_PATH)
y_test = np.load(Y_TEST_PATH)
print("Data loaded successfully.")

# Create Dataset instances USING THE NEW CLASS
train_dataset = UltrasoundNpyDataset_NoTransforms(x_train, y_train)
val_dataset = UltrasoundNpyDataset_NoTransforms(x_val, y_val)
test_dataset = UltrasoundNpyDataset_NoTransforms(x_test, y_test)

# Create DataLoader instances
BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nCreated {len(train_dataset)} training examples (without transformations).")
print(f"Created {len(val_dataset)} validation examples (without transformations).")
print(f"Created {len(test_dataset)} testing examples (without transformations).")
print("Data preparation complete.")


Loading pre-split data...
Data loaded successfully.

Created 149 training examples (without transformations).
Created 32 validation examples (without transformations).
Created 33 testing examples (without transformations).
Data preparation complete.


In [19]:
# Step 3: Define the Attention U-Net Model

# --- 3.1: Set up the device (GPU or CPU) ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# --- 3.2: Create the Attention U-Net Model ---
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=1,              # Match dataset (grayscale)
    classes=1,
    attention_type='scse'
)

model.to(DEVICE)

print("Attention U-Net model created successfully.")


Using device: cuda
Attention U-Net model created successfully.


In [20]:
# --- 4.1: Loss Function ---
class CombinedLoss(nn.Module):
    def __init__(self):
        super(CombinedLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, preds, targets):
        bce_loss = self.bce(preds, targets)
        preds_sigmoid = torch.sigmoid(preds)
        intersection = (preds_sigmoid * targets).sum()
        union = preds_sigmoid.sum() + targets.sum()
        dice_loss = 1 - (2. * intersection + 1e-6) / (union + 1e-6)
        return bce_loss + dice_loss

loss_fn = CombinedLoss()

# --- 4.2: Optimizer and Scheduler ---
LEARNING_RATE = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# --- 4.3: Metrics Calculation (Dice Score) ---
def dice_score(preds, targets, smooth=1e-6):
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()
    preds = preds.view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    return (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)

print("Loss function, optimizer, and metrics are configured.")

Loss function, optimizer, and metrics are configured.


In [22]:
# --- 5.1: Training Loop ---
NUM_EPOCHS = 100
best_val_dice = 0.0
patience = 10
trigger_times = 0
MODEL_SAVE_PATH = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/attentionunet_resnet34_best.pth'

for epoch in range(NUM_EPOCHS):
    print(f"--- Epoch {epoch+1}/{NUM_EPOCHS} ---")

    # --- Training Phase ---
    model.train()
    train_loss = 0.0
    train_dice = 0.0
    for images, masks in tqdm(train_loader, desc="Training"):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        if masks.dim() != outputs.dim() or masks.size() != outputs.size():
            masks = masks.squeeze(-1)  # Remove extra dimension if present
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_dice += dice_score(outputs, masks).item()

    avg_train_loss = train_loss / len(train_loader)
    avg_train_dice = train_dice / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    with torch.no_grad():
        for i, (images, masks) in enumerate(tqdm(val_loader, desc="Validation")):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            if masks.dim() != outputs.dim() or masks.size() != outputs.size():
                masks = masks.squeeze(-1)  # Remove extra dimension if present
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()
            val_dice += dice_score(outputs, masks).item()
            # Debug: Check the first image of the first batch
            if i == 0:  # Check first batch
                for j in range(images.size(0)):  # Iterate over batch size
                    if j == 0:  # Check first image in batch
                        sample_pred = torch.sigmoid(outputs[j]).cpu().numpy()
                        print(f"Epoch {epoch+1}, Batch {i}, Image {j} Prediction Range: min={sample_pred.min()}, max={sample_pred.max()}")

    avg_val_loss = val_loss / len(val_loader)
    avg_val_dice = val_dice / len(val_loader)

    scheduler.step(avg_val_dice)

    if avg_val_dice > best_val_dice:
        best_val_dice = avg_val_dice
        trigger_times = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"New best model saved at epoch {epoch+1} with Val Dice: {avg_val_dice:.4f} to {MODEL_SAVE_PATH}")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping triggered at epoch {epoch+1} with no improvement.")
            break

    print(f"Epoch {epoch+1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}, Train Dice: {avg_train_dice:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}, Val Dice:   {avg_val_dice:.4f}\n")

print("Training finished!")
print(f"Best model was saved with Validation Dice: {best_val_dice:.4f} to {MODEL_SAVE_PATH}")

--- Epoch 1/100 ---


Training:   0%|          | 0/19 [00:00<?, ?it/s]

Validation:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1, Batch 0, Image 0 Prediction Range: min=0.08298218995332718, max=0.9977005124092102
New best model saved at epoch 1 with Val Dice: 0.7865 to /content/drive/MyDrive/internship models/attention unet model/rectus femoris/attentionunet_resnet34_best.pth
Epoch 1 Summary:
  Train Loss: 0.8590, Train Dice: 0.8233
  Val Loss:   0.8512, Val Dice:   0.7865

--- Epoch 2/100 ---


Training:   0%|          | 0/19 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 31, Batch 0, Image 0 Prediction Range: min=0.008570403791964054, max=0.9997561573982239
Epoch 31 Summary:
  Train Loss: 0.0942, Train Dice: 0.9883
  Val Loss:   0.1848, Val Dice:   0.9426

--- Epoch 32/100 ---


Training:   0%|          | 0/19 [00:00<?, ?it/s]

Validation:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 32, Batch 0, Image 0 Prediction Range: min=0.008456176146864891, max=0.9997187256813049
Early stopping triggered at epoch 32 with no improvement.
Training finished!
Best model was saved with Validation Dice: 0.9434 to /content/drive/MyDrive/internship models/attention unet model/rectus femoris/attentionunet_resnet34_best.pth


In [26]:
# New step 6
# --- 6.1: Import necessary libraries ---
import os
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import segmentation_models_pytorch as smp

# --- 6.2: Define the Post-Processing Function (Unchanged) ---
def post_process_mask(mask):
    """
    Applies post-processing to a binary segmentation mask.
    This function finds all disconnected objects (connected components) and keeps only the largest one,
    then fills any holes within that largest object.
    """
    labels, num_features = ndimage.label(mask)
    if num_features == 0:
        return mask

    component_sizes = np.bincount(labels.ravel())
    if len(component_sizes) > 1:
        largest_component_label = component_sizes[1:].argmax() + 1
        processed_mask = (labels == largest_component_label)
        processed_mask = ndimage.binary_fill_holes(processed_mask)
        return processed_mask.astype(np.uint8)
    else:
        return mask

# --- 6.3: Load the Best Model (Updated) ---
print("Loading the best model for evaluation...")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,  # Weights are loaded from file, not pre-trained
    in_channels=1,         # Match the trained Attention U-Net model
    classes=1,
    attention_type='scse'  # Use Attention U-Net
)
MODEL_SAVE_PATH = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/attentionunet_resnet34_best.pth'
eval_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=torch.device(DEVICE)))
eval_model.to(DEVICE)
eval_model.eval()

# --- 6.4: Define Save Directories and Visualization Helper ---
BASE_SAVE_DIR = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/segmentation_results_with_preprocessing'
TRAIN_SAVE_DIR = os.path.join(BASE_SAVE_DIR, 'train_set_predictions')
TEST_SAVE_DIR = os.path.join(BASE_SAVE_DIR, 'test_set_predictions')

os.makedirs(TRAIN_SAVE_DIR, exist_ok=True)
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

print(f"Train set predictions will be saved to: {TRAIN_SAVE_DIR}")
print(f"Test set predictions will be saved to: {TEST_SAVE_DIR}")

def visualize_and_save(processed_img, gt_mask, pred_raw, pred_post, save_path, title):
    """Helper function to plot and save comparison images."""
    if processed_img.shape[0] == 1:  # Grayscale
        processed_img_display = processed_img.cpu().squeeze(0).numpy()
        processed_img_display = (processed_img_display - processed_img_display.min()) / (processed_img_display.max() - processed_img_display.min())
    else:  # RGB (repeated channel) - unlikely now with in_channels=1
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        processed_img_display = processed_img.cpu().permute(1, 2, 0).numpy()
        processed_img_display = std * processed_img_display + mean
        processed_img_display = np.clip(processed_img_display, 0, 1)

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(processed_img_display, cmap='gray')
    axes[0].set_title("Original Black-White Input")
    axes[0].axis('off')
    axes[1].imshow(np.squeeze(gt_mask), cmap='gray')
    axes[1].set_title("Ground Truth")
    axes[1].axis('off')
    axes[2].imshow(np.squeeze(pred_raw), cmap='gray')
    axes[2].set_title("Raw Prediction (Before Post-Processing)")
    axes[2].axis('off')
    axes[3].imshow(np.squeeze(pred_post), cmap='gray')
    axes[3].set_title("Post-Processed Prediction")
    axes[3].axis('off')
    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the figure to save memory

# --- 6.5: Run Evaluation on the ENTIRE Test Set & Save ALL Predictions ---
print("\n--- Evaluating ENTIRE Test Set and Saving All Predictions ---")
total_dice_before = 0
total_dice_after = 0
num_samples_test = 0
smooth = 1e-6

with torch.no_grad():
    for i, (images, gt_masks) in enumerate(tqdm(test_loader, desc="Testing and Saving")):
        images = images.to(DEVICE)
        gt_masks_np = gt_masks.cpu().numpy()

        preds_logits = eval_model(images)
        preds_sigmoid = torch.sigmoid(preds_logits)
        preds_before_np = (preds_sigmoid > 0.5).cpu().numpy()
        preds_after_np = np.array([post_process_mask(np.squeeze(p)) for p in preds_before_np])

        # Loop through each image in the batch to save it
        for j in range(images.shape[0]):
            image_idx = i * test_loader.batch_size + j

            gt = np.squeeze(gt_masks_np[j]).flatten()
            pred_before = np.squeeze(preds_before_np[j]).flatten()
            pred_after = np.squeeze(preds_after_np[j]).flatten()

            intersection_before = (pred_before * gt).sum()
            total_dice_before += (2. * intersection_before + smooth) / (pred_before.sum() + gt.sum() + smooth)

            intersection_after = (pred_after * gt).sum()
            total_dice_after += (2. * intersection_after + smooth) / (pred_after.sum() + gt.sum() + smooth)

            num_samples_test += 1

            # Define the save path for this specific image
            save_path = os.path.join(TEST_SAVE_DIR, f"test_prediction_{image_idx+1}.png")

            # Call the helper function to save the plot
            visualize_and_save(
                processed_img=images[j],
                gt_mask=gt_masks_np[j],
                pred_raw=preds_before_np[j],
                pred_post=preds_after_np[j],
                save_path=save_path,
                title=f"Test Set - Prediction {image_idx+1}"
            )

# --- 6.6: Print Final Test Results ---
avg_dice_before_test = total_dice_before / num_samples_test
avg_dice_after_test = total_dice_after / num_samples_test
print(f"\n--- Test Set Evaluation Complete ---")
print(f"Total Test Images Processed: {num_samples_test}")
print(f"Average Dice (Before Post-Processing): {avg_dice_before_test:.4f}")
print(f"Average Dice (After Post-Processing):  {avg_dice_after_test:.4f}")

# --- 6.7: Run Prediction on the ENTIRE Train Set & Save ALL Predictions ---
print("\n--- Evaluating ENTIRE Train Set and Saving All Predictions ---")
num_samples_train = 0
total_dice_before_train = 0
total_dice_after_train = 0

with torch.no_grad():
    for i, (images, gt_masks) in enumerate(tqdm(train_loader, desc="Training Set Prediction")):
        images = images.to(DEVICE)
        gt_masks_np = gt_masks.cpu().numpy()

        preds_logits = eval_model(images)
        preds_sigmoid = torch.sigmoid(preds_logits)
        preds_before_np = (preds_sigmoid > 0.5).cpu().numpy()
        preds_after_np = np.array([post_process_mask(np.squeeze(p)) for p in preds_before_np])

        # Loop through each image in the batch to save it
        for j in range(images.shape[0]):
            image_idx = i * train_loader.batch_size + j

            gt = np.squeeze(gt_masks_np[j]).flatten()
            pred_before = np.squeeze(preds_before_np[j]).flatten()
            pred_after = np.squeeze(preds_after_np[j]).flatten()

            intersection_before = (pred_before * gt).sum()
            total_dice_before_train += (2. * intersection_before + smooth) / (pred_before.sum() + gt.sum() + smooth)

            intersection_after = (pred_after * gt).sum()
            total_dice_after_train += (2. * intersection_after + smooth) / (pred_after.sum() + gt.sum() + smooth)

            num_samples_train += 1

            # Define the save path for this specific image
            save_path = os.path.join(TRAIN_SAVE_DIR, f"train_prediction_{image_idx+1}.png")

            # Call the helper function to save the plot
            visualize_and_save(
                processed_img=images[j],
                gt_mask=gt_masks_np[j],
                pred_raw=preds_before_np[j],
                pred_post=preds_after_np[j],
                save_path=save_path,
                title=f"Train Set - Prediction {image_idx+1}"
            )

# --- 6.8: Print Final Train Results ---
avg_dice_before_train = total_dice_before_train / num_samples_train
avg_dice_after_train = total_dice_after_train / num_samples_train
print(f"\n--- Train Set Evaluation Complete ---")
print(f"Total Train Images Processed: {num_samples_train}")
print(f"Average Dice (Before Post-Processing): {avg_dice_before_train:.4f}")
print(f"Average Dice (After Post-Processing):  {avg_dice_after_train:.4f}")
print(f"All predictions saved successfully to Google Drive.")

Loading the best model for evaluation...
Train set predictions will be saved to: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/segmentation_results_with_preprocessing/train_set_predictions
Test set predictions will be saved to: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/segmentation_results_with_preprocessing/test_set_predictions

--- Evaluating ENTIRE Test Set and Saving All Predictions ---


Testing and Saving: 100%|██████████| 5/5 [00:37<00:00,  7.59s/it]



--- Test Set Evaluation Complete ---
Total Test Images Processed: 33
Average Dice (Before Post-Processing): 0.9305
Average Dice (After Post-Processing):  0.9305

--- Evaluating ENTIRE Train Set and Saving All Predictions ---


Training Set Prediction: 100%|██████████| 19/19 [02:50<00:00,  8.96s/it]


--- Train Set Evaluation Complete ---
Total Train Images Processed: 149
Average Dice (Before Post-Processing): 0.9820
Average Dice (After Post-Processing):  0.9820
All predictions saved successfully to Google Drive.


# no need la

In [ ]:
# Step 7: Final Evaluation on the Test Set

print("--- Evaluating model on the test set ---")

model.eval()
test_loss = 0.0
test_dice = 0.0

with torch.no_grad():
    for images, masks in tqdm(test_loader, desc="Testing"):
        images = images.to(DEVICE)
        masks = masks.float().to(DEVICE)

        outputs = model(images)
        loss = loss_fn(outputs, masks)

        test_loss += loss.item()
        test_dice += dice_score(outputs, masks).item()

avg_test_loss = test_loss / len(test_loader)
avg_test_dice = test_dice / len(test_loader)

print("\n--- Final Test Set Performance ---")
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test Dice Score: {avg_test_dice:.4f}")


--- Evaluating model on the test set ---


Testing:   0%|          | 0/5 [00:00<?, ?it/s]


--- Final Test Set Performance ---
  Test Loss: 0.0672
  Test Dice Score: 0.9410


In [ ]:
# Step 8: Create all output directories

import os

# Define a base path for your project's outputs
DRIVE_BASE_PATH = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/'

# Define specific directories for training and testing image samples
TRAIN_SAVE_DIR = os.path.join(DRIVE_BASE_PATH, 'internal validation')
TEST_SAVE_DIR = os.path.join(DRIVE_BASE_PATH, 'external validation')

# Create all directories. 'exist_ok=True' prevents errors if they already exist.
os.makedirs(TRAIN_SAVE_DIR, exist_ok=True)
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

print(f"Created and verified directory for training samples: {TRAIN_SAVE_DIR}")
print(f"Created and verified directory for test results: {TEST_SAVE_DIR}")


Created and verified directory for training samples: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/internal validation
Created and verified directory for test results: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/external validation


In [ ]:
# Step 9: Generate and Save All Post-Processed Masks

# --- 1. Import necessary libraries ---
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import ndimage
import segmentation_models_pytorch as smp # Make sure smp is imported

print("--- Starting mask generation process ---")

# --- 2. Define Directories ---
# A main directory to hold all generated masks
MASKS_BASE_DIR = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/'

# Specific subdirectories for train and test sets
TRAIN_MASKS_SAVE_DIR = os.path.join(MASKS_BASE_DIR, 'internal validation')
TEST_MASKS_SAVE_DIR = os.path.join(MASKS_BASE_DIR, 'external validation')

# Create the directories if they don't exist
os.makedirs(TRAIN_MASKS_SAVE_DIR, exist_ok=True)
os.makedirs(TEST_MASKS_SAVE_DIR, exist_ok=True)

print(f"Train masks will be saved to: {TRAIN_MASKS_SAVE_DIR}")
print(f"Test masks will be saved to: {TEST_MASKS_SAVE_DIR}")

# --- 3. Load the Best Model ---
# Ensure DEVICE is defined (e.g., DEVICE = "cuda" if torch.cuda.is_available() else "cpu")
print("\nLoading the best model weights...")
model = smp.Unet(
    encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=1, attention_type='scse'
)
MODEL_SAVE_PATH = '/content/drive/MyDrive/internship models/attention unet model/rectus femoris/attentionunet_resnet34_best.pth'
model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval() # IMPORTANT: Set model to evaluation mode

# --- 4. Define the Post-Processing Function ---
def post_process_mask(mask):
    """A 2D binary numpy array (H, W) representing the predicted mask."""
    labels, num_features = ndimage.label(mask)
    if num_features == 0: return mask
    component_sizes = np.bincount(labels.ravel())
    if len(component_sizes) > 1:
        largest_component_label = component_sizes[1:].argmax() + 1
        processed_mask = (labels == largest_component_label)
        processed_mask = ndimage.binary_fill_holes(processed_mask)
        return processed_mask.astype(np.uint8)
    else:
        return mask

# --- 5. Process and Save Masks for the ENTIRE Training Set ---
print(f"\nGenerating masks for {len(train_dataset)} training images...")
train_mask_counter = 0
# Use torch.no_grad() to speed up inference and reduce memory usage
with torch.no_grad():
    # Iterate through the train_loader with a progress bar
    for images, _ in tqdm(train_loader, desc="Processing Training Set"):
        images = images.to(DEVICE)

        # Get model predictions
        preds_logits = model(images)
        preds_binary = (torch.sigmoid(preds_logits) > 0.5).cpu().numpy()

        # Loop through each mask in the batch
        for i in range(preds_binary.shape[0]):
            raw_mask = np.squeeze(preds_binary[i])
            processed_mask = post_process_mask(raw_mask)

            # Define the save path. Using a counter for a unique name.
            save_path = os.path.join(TRAIN_MASKS_SAVE_DIR, f"train_mask_{train_mask_counter + 1}.png")

            # Save the numpy array as a grayscale image.
            # plt.imsave is perfect for this as it doesn't add axes or padding.
            plt.imsave(save_path, processed_mask, cmap='gray')

            train_mask_counter += 1

print(f"Successfully saved {train_mask_counter} processed masks from the training set.")


# --- 6. Process and Save Masks for the ENTIRE Test Set ---
print(f"\nGenerating masks for {len(test_dataset)} test images...")
test_mask_counter = 0
with torch.no_grad():
    for images, _ in tqdm(test_loader, desc="Processing Test Set"):
        images = images.to(DEVICE)

        preds_logits = model(images)
        preds_binary = (torch.sigmoid(preds_logits) > 0.5).cpu().numpy()

        for i in range(preds_binary.shape[0]):
            raw_mask = np.squeeze(preds_binary[i])
            processed_mask = post_process_mask(raw_mask)

            save_path = os.path.join(TEST_MASKS_SAVE_DIR, f"test_mask_{test_mask_counter + 1}.png")
            plt.imsave(save_path, processed_mask, cmap='gray')

            test_mask_counter += 1

print(f"Successfully saved {test_mask_counter} processed masks from the test set.")
print("\n--- All tasks complete! ---")


--- Starting mask generation process ---
Train masks will be saved to: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/internal validation
Test masks will be saved to: /content/drive/MyDrive/internship models/attention unet model/rectus femoris/external validation

Loading the best model weights...

Generating masks for 149 training images...


Processing Training Set: 100%|██████████| 19/19 [00:04<00:00,  4.24it/s]


Successfully saved 149 processed masks from the training set.

Generating masks for 33 test images...


Processing Test Set: 100%|██████████| 5/5 [00:00<00:00,  7.54it/s]

Successfully saved 33 processed masks from the test set.

--- All tasks complete! ---


# vastus medialis muscle

In [27]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# ========================================================================
# This is the simplified Dataset class without transformations
# ========================================================================

class UltrasoundNpyDataset_NoTransforms(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        image_np = self.x_data[idx]
        mask_np = self.y_data[idx]

        # Normalize image to [0, 1] based on min/max of the dataset
        image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min() + 1e-6)
        image_tensor = torch.from_numpy(image_np).float()
        if image_tensor.ndim == 3 and image_tensor.shape[-1] in [1, 3]:
            image_tensor = image_tensor.permute(2, 0, 1)
        elif image_tensor.ndim == 2:
            image_tensor = image_tensor.unsqueeze(0)

        # Process mask: ensure it’s 2D (H, W) and convert to binary float
        if mask_np.ndim == 3 and mask_np.shape[-1] == 1:
            mask_np = np.squeeze(mask_np, axis=-1)  # Remove the last dimension if it’s 1
        mask_np = (mask_np > 0).astype(np.float32)  # Binary mask (0 or 1)
        mask_tensor = torch.from_numpy(mask_np).float()
        if mask_tensor.ndim == 2:
            mask_tensor = mask_tensor.unsqueeze(0)  # Add channel dimension to [1, H, W]

        return image_tensor, mask_tensor


# ========================================================================
# The rest of your script, now using the new Dataset class
# ========================================================================

# --- Step 1: Define File Paths ---
# This path points to the folder containing your data.
DATA_FOLDER = '/content/drive/MyDrive/intern RF longitudinal latest file/'

X_TRAIN_PATH = os.path.join(DATA_FOLDER, 'X_train.npy')
Y_TRAIN_PATH = os.path.join(DATA_FOLDER, 'y_train.npy')
X_VAL_PATH = os.path.join(DATA_FOLDER, 'X_val.npy')
Y_VAL_PATH = os.path.join(DATA_FOLDER, 'y_val.npy')
X_TEST_PATH = os.path.join(DATA_FOLDER, 'X_test.npy')
Y_TEST_PATH = os.path.join(DATA_FOLDER, 'y_test.npy')


# --- Step 2: Load Data and Create DataLoaders ---
print("Loading pre-split data...")
x_train = np.load(X_TRAIN_PATH)
y_train = np.load(Y_TRAIN_PATH)
x_val = np.load(X_VAL_PATH)
y_val = np.load(Y_VAL_PATH)
x_test = np.load(X_TEST_PATH)
y_test = np.load(Y_TEST_PATH)
print("Data loaded successfully.")

# Create Dataset instances USING THE NEW CLASS
train_dataset = UltrasoundNpyDataset_NoTransforms(x_train, y_train)
val_dataset = UltrasoundNpyDataset_NoTransforms(x_val, y_val)
test_dataset = UltrasoundNpyDataset_NoTransforms(x_test, y_test)

# Create DataLoader instances
BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\nCreated {len(train_dataset)} training examples (without transformations).")
print(f"Created {len(val_dataset)} validation examples (without transformations).")
print(f"Created {len(test_dataset)} testing examples (without transformations).")
print("Data preparation complete.")


Loading pre-split data...
Data loaded successfully.

Created 126 training examples (without transformations).
Created 27 validation examples (without transformations).
Created 27 testing examples (without transformations).
Data preparation complete.


In [29]:
# --- 5.1: Training Loop ---
NUM_EPOCHS = 50
best_val_dice = 0.0
patience = 50
trigger_times = 0
MODEL_SAVE_PATH = '/content/drive/MyDrive/internship models/attention unet model/vastus medialis/attentionunet_resnet34_best.pth'

for epoch in range(NUM_EPOCHS):
    print(f"--- Epoch {epoch+1}/{NUM_EPOCHS} ---")

    # --- Training Phase ---
    model.train()
    train_loss = 0.0
    train_dice = 0.0
    for images, masks in tqdm(train_loader, desc="Training"):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        if masks.dim() != outputs.dim() or masks.size() != outputs.size():
            masks = masks.squeeze(-1)  # Remove extra dimension if present
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_dice += dice_score(outputs, masks).item()

    avg_train_loss = train_loss / len(train_loader)
    avg_train_dice = train_dice / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    with torch.no_grad():
        for i, (images, masks) in enumerate(tqdm(val_loader, desc="Validation")):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)
            if masks.dim() != outputs.dim() or masks.size() != outputs.size():
                masks = masks.squeeze(-1)  # Remove extra dimension if present
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()
            val_dice += dice_score(outputs, masks).item()
            # Debug: Check the first image of the first batch
            if i == 0:  # Check first batch
                for j in range(images.size(0)):  # Iterate over batch size
                    if j == 0:  # Check first image in batch
                        sample_pred = torch.sigmoid(outputs[j]).cpu().numpy()
                        print(f"Epoch {epoch+1}, Batch {i}, Image {j} Prediction Range: min={sample_pred.min()}, max={sample_pred.max()}")

    avg_val_loss = val_loss / len(val_loader)
    avg_val_dice = val_dice / len(val_loader)

    scheduler.step(avg_val_dice)

    if avg_val_dice > best_val_dice:
        best_val_dice = avg_val_dice
        trigger_times = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"New best model saved at epoch {epoch+1} with Val Dice: {avg_val_dice:.4f} to {MODEL_SAVE_PATH}")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping triggered at epoch {epoch+1} with no improvement.")
            break

    print(f"Epoch {epoch+1} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f}, Train Dice: {avg_train_dice:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}, Val Dice:   {avg_val_dice:.4f}\n")

print("Training finished!")
print(f"Best model was saved with Validation Dice: {best_val_dice:.4f} to {MODEL_SAVE_PATH}")

--- Epoch 1/50 ---


Validation: 100%|██████████| 4/4 [00:00<00:00, 31.50it/s]


Epoch 1, Batch 0, Image 0 Prediction Range: min=0.0037485852371901274, max=0.9981468915939331
New best model saved at epoch 1 with Val Dice: 0.9273 to /content/drive/MyDrive/internship models/attention unet model/vastus medialis/attentionunet_resnet34_best.pth
Epoch 1 Summary:
  Train Loss: 0.0760, Train Dice: 0.9817
  Val Loss:   0.2360, Val Dice:   0.9273

--- Epoch 2/50 ---


Validation: 100%|██████████| 4/4 [00:00<00:00, 30.86it/s]


Epoch 2, Batch 0, Image 0 Prediction Range: min=0.0036763600073754787, max=0.9980316758155823
Epoch 2 Summary:
  Train Loss: 0.0752, Train Dice: 0.9819
  Val Loss:   0.2397, Val Dice:   0.9264

--- Epoch 3/50 ---


...

Epoch 48, Batch 0, Image 0 Prediction Range: min=0.0036760070361196995, max=0.9982184767723083
Epoch 48 Summary:
  Train Loss: 0.0660, Train Dice: 0.9852
  Val Loss:   0.2413, Val Dice:   0.9268

--- Epoch 49/50 ---


Validation: 100%|██████████| 4/4 [00:00<00:00, 31.28it/s]


Epoch 49, Batch 0, Image 0 Prediction Range: min=0.0036687629763036966, max=0.9982094764709473
Epoch 49 Summary:
  Train Loss: 0.0646, Train Dice: 0.9856
  Val Loss:   0.2416, Val Dice:   0.9266

--- Epoch 50/50 ---


Validation: 100%|██████████| 4/4 [00:00<00:00, 30.08it/s]

Epoch 50, Batch 0, Image 0 Prediction Range: min=0.0036449634935706854, max=0.9981671571731567
Epoch 50 Summary:
  Train Loss: 0.0646, Train Dice: 0.9856
  Val Loss:   0.2431, Val Dice:   0.9263

Training finished!
Best model was saved with Validation Dice: 0.9284 to /content/drive/MyDrive/internship models/attention unet model/vastus medialis/attentionunet_resnet34_best.pth


In [30]:
# New step 6
# --- 6.1: Import necessary libraries ---
import os
from scipy import ndimage
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import segmentation_models_pytorch as smp

# --- 6.2: Define the Post-Processing Function (Unchanged) ---
def post_process_mask(mask):
    """
    Applies post-processing to a binary segmentation mask.
    This function finds all disconnected objects (connected components) and keeps only the largest one,
    then fills any holes within that largest object.
    """
    labels, num_features = ndimage.label(mask)
    if num_features == 0:
        return mask

    component_sizes = np.bincount(labels.ravel())
    if len(component_sizes) > 1:
        largest_component_label = component_sizes[1:].argmax() + 1
        processed_mask = (labels == largest_component_label)
        processed_mask = ndimage.binary_fill_holes(processed_mask)
        return processed_mask.astype(np.uint8)
    else:
        return mask

# --- 6.3: Load the Best Model (Updated) ---
print("Loading the best model for evaluation...")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,  # Weights are loaded from file, not pre-trained
    in_channels=1,         # Match the trained Attention U-Net model
    classes=1,
    attention_type='scse'  # Use Attention U-Net
)
MODEL_SAVE_PATH = '/content/drive/MyDrive/internship models/attention unet model/vastus medialis/attentionunet_resnet34_best.pth'
eval_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=torch.device(DEVICE)))
eval_model.to(DEVICE)
eval_model.eval()

# --- 6.4: Define Save Directories and Visualization Helper ---
BASE_SAVE_DIR = '/content/drive/MyDrive/internship models/attention unet model/vastus medialis/segmentation_results_with_preprocessing'
TRAIN_SAVE_DIR = os.path.join(BASE_SAVE_DIR, 'train_set_predictions')
TEST_SAVE_DIR = os.path.join(BASE_SAVE_DIR, 'test_set_predictions')

os.makedirs(TRAIN_SAVE_DIR, exist_ok=True)
os.makedirs(TEST_SAVE_DIR, exist_ok=True)

print(f"Train set predictions will be saved to: {TRAIN_SAVE_DIR}")
print(f"Test set predictions will be saved to: {TEST_SAVE_DIR}")

def visualize_and_save(processed_img, gt_mask, pred_raw, pred_post, save_path, title):
    """Helper function to plot and save comparison images."""
    if processed_img.shape[0] == 1:  # Grayscale
        processed_img_display = processed_img.cpu().squeeze(0).numpy()
        processed_img_display = (processed_img_display - processed_img_display.min()) / (processed_img_display.max() - processed_img_display.min())
    else:  # RGB (repeated channel) - unlikely now with in_channels=1
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        processed_img_display = processed_img.cpu().permute(1, 2, 0).numpy()
        processed_img_display = std * processed_img_display + mean
        processed_img_display = np.clip(processed_img_display, 0, 1)

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    axes[0].imshow(processed_img_display, cmap='gray')
    axes[0].set_title("Original Black-White Input")
    axes[0].axis('off')
    axes[1].imshow(np.squeeze(gt_mask), cmap='gray')
    axes[1].set_title("Ground Truth")
    axes[1].axis('off')
    axes[2].imshow(np.squeeze(pred_raw), cmap='gray')
    axes[2].set_title("Raw Prediction (Before Post-Processing)")
    axes[2].axis('off')
    axes[3].imshow(np.squeeze(pred_post), cmap='gray')
    axes[3].set_title("Post-Processed Prediction")
    axes[3].axis('off')
    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the figure to save memory

# --- 6.5: Run Evaluation on the ENTIRE Test Set & Save ALL Predictions ---
print("\n--- Evaluating ENTIRE Test Set and Saving All Predictions ---")
total_dice_before = 0
total_dice_after = 0
num_samples_test = 0
smooth = 1e-6

with torch.no_grad():
    for i, (images, gt_masks) in enumerate(tqdm(test_loader, desc="Testing and Saving")):
        images = images.to(DEVICE)
        gt_masks_np = gt_masks.cpu().numpy()

        preds_logits = eval_model(images)
        preds_sigmoid = torch.sigmoid(preds_logits)
        preds_before_np = (preds_sigmoid > 0.5).cpu().numpy()
        preds_after_np = np.array([post_process_mask(np.squeeze(p)) for p in preds_before_np])

        # Loop through each image in the batch to save it
        for j in range(images.shape[0]):
            image_idx = i * test_loader.batch_size + j

            gt = np.squeeze(gt_masks_np[j]).flatten()
            pred_before = np.squeeze(preds_before_np[j]).flatten()
            pred_after = np.squeeze(preds_after_np[j]).flatten()

            intersection_before = (pred_before * gt).sum()
            total_dice_before += (2. * intersection_before + smooth) / (pred_before.sum() + gt.sum() + smooth)

            intersection_after = (pred_after * gt).sum()
            total_dice_after += (2. * intersection_after + smooth) / (pred_after.sum() + gt.sum() + smooth)

            num_samples_test += 1

            # Define the save path for this specific image
            save_path = os.path.join(TEST_SAVE_DIR, f"test_prediction_{image_idx+1}.png")

            # Call the helper function to save the plot
            visualize_and_save(
                processed_img=images[j],
                gt_mask=gt_masks_np[j],
                pred_raw=preds_before_np[j],
                pred_post=preds_after_np[j],
                save_path=save_path,
                title=f"Test Set - Prediction {image_idx+1}"
            )

# --- 6.6: Print Final Test Results ---
avg_dice_before_test = total_dice_before / num_samples_test
avg_dice_after_test = total_dice_after / num_samples_test
print(f"\n--- Test Set Evaluation Complete ---")
print(f"Total Test Images Processed: {num_samples_test}")
print(f"Average Dice (Before Post-Processing): {avg_dice_before_test:.4f}")
print(f"Average Dice (After Post-Processing):  {avg_dice_after_test:.4f}")

# --- 6.7: Run Prediction on the ENTIRE Train Set & Save ALL Predictions ---
print("\n--- Evaluating ENTIRE Train Set and Saving All Predictions ---")
num_samples_train = 0
total_dice_before_train = 0
total_dice_after_train = 0

with torch.no_grad():
    for i, (images, gt_masks) in enumerate(tqdm(train_loader, desc="Training Set Prediction")):
        images = images.to(DEVICE)
        gt_masks_np = gt_masks.cpu().numpy()

        preds_logits = eval_model(images)
        preds_sigmoid = torch.sigmoid(preds_logits)
        preds_before_np = (preds_sigmoid > 0.5).cpu().numpy()
        preds_after_np = np.array([post_process_mask(np.squeeze(p)) for p in preds_before_np])

        # Loop through each image in the batch to save it
        for j in range(images.shape[0]):
            image_idx = i * train_loader.batch_size + j

            gt = np.squeeze(gt_masks_np[j]).flatten()
            pred_before = np.squeeze(preds_before_np[j]).flatten()
            pred_after = np.squeeze(preds_after_np[j]).flatten()

            intersection_before = (pred_before * gt).sum()
            total_dice_before_train += (2. * intersection_before + smooth) / (pred_before.sum() + gt.sum() + smooth)

            intersection_after = (pred_after * gt).sum()
            total_dice_after_train += (2. * intersection_after + smooth) / (pred_after.sum() + gt.sum() + smooth)

            num_samples_train += 1

            # Define the save path for this specific image
            save_path = os.path.join(TRAIN_SAVE_DIR, f"train_prediction_{image_idx+1}.png")

            # Call the helper function to save the plot
            visualize_and_save(
                processed_img=images[j],
                gt_mask=gt_masks_np[j],
                pred_raw=preds_before_np[j],
                pred_post=preds_after_np[j],
                save_path=save_path,
                title=f"Train Set - Prediction {image_idx+1}"
            )

# --- 6.8: Print Final Train Results ---
avg_dice_before_train = total_dice_before_train / num_samples_train
avg_dice_after_train = total_dice_after_train / num_samples_train
print(f"\n--- Train Set Evaluation Complete ---")
print(f"Total Train Images Processed: {num_samples_train}")
print(f"Average Dice (Before Post-Processing): {avg_dice_before_train:.4f}")
print(f"Average Dice (After Post-Processing):  {avg_dice_after_train:.4f}")
print(f"All predictions saved successfully to Google Drive.")

Loading the best model for evaluation...
Train set predictions will be saved to: /content/drive/MyDrive/internship models/attention unet model/vastus medialis/segmentation_results_with_preprocessing/train_set_predictions
Test set predictions will be saved to: /content/drive/MyDrive/internship models/attention unet model/vastus medialis/segmentation_results_with_preprocessing/test_set_predictions

--- Evaluating ENTIRE Test Set and Saving All Predictions ---


Testing and Saving: 100%|██████████| 4/4 [00:31<00:00,  7.92s/it]



--- Test Set Evaluation Complete ---
Total Test Images Processed: 27
Average Dice (Before Post-Processing): 0.9371
Average Dice (After Post-Processing):  0.9375

--- Evaluating ENTIRE Train Set and Saving All Predictions ---


Training Set Prediction: 100%|██████████| 16/16 [02:20<00:00,  8.78s/it]


--- Train Set Evaluation Complete ---
Total Train Images Processed: 126
Average Dice (Before Post-Processing): 0.9852
Average Dice (After Post-Processing):  0.9852
All predictions saved successfully to Google Drive.
